In [1]:
import pandas as pd 
import numpy as np
from incf.countryutils import transformations

In [2]:
medals = pd.read_csv('Olympic_Medals.csv')
IOC_cont = pd.read_csv('IOC to continent.csv')

In [3]:
#Columns Headers
medals.columns.values 

array(['City', 'Edition', 'Sport', 'Discipline', 'Athlete', 'NOC',
       'Gender', 'Event', 'Event_gender', 'Medal'], dtype=object)

In [4]:
OM = medals[['NOC','Gender','Event_gender','Medal', 'Edition']]

In [5]:
#Columns Headers
OM.columns.values

array(['NOC', 'Gender', 'Event_gender', 'Medal', 'Edition'], dtype=object)

In [6]:
#Percentage Table that gives % of medals won by country that are won (male versus female)
coun_gend = OM.groupby(['Edition','NOC','Gender']).count()[['Medal']] \
              .unstack().fillna(0).apply(lambda x: 100*x/float(x.sum()), axis=1) \
              .stack().reset_index()
#Same as above but index has not been set and its a count not a %
t2 = OM.groupby(['Edition','NOC','Gender']).count()[['Medal']].unstack().fillna(0).apply(lambda x: 100*x/float(x.sum()), axis=1)
t2 = OM.groupby(['Edition','NOC','Gender']).count()[['Medal']].unstack().fillna(0)
# same as above but flatened into a df
event= t2.stack().reset_index()
event

,Edition,NOC,Gender,Medal
0,1896,AUS,Men,2
1,1896,AUS,Women,0
2,1896,AUT,Men,5
3,1896,AUT,Women,0
4,1896,DEN,Men,6
5,1896,DEN,Women,0
6,1896,FRA,Men,11
7,1896,FRA,Women,0
8,1896,GBR,Men,7
9,1896,GBR,Women,0


In [9]:
#uses t2 above to count the total amount of events possible to win (male verus female
event_year = t2.groupby(level=0).apply(lambda x: x.sum()) \
               .apply(lambda x: 100*x/float(x.sum()), axis = 1) \
               .unstack().reset_index()
event_year = event_year.drop('level_0', axis=1)

#merges tables together to create one table
medal_table=pd.merge(coun_gend, event_year, how='inner', on=['Edition','Gender'])

#seperates all men from table into two tables and then merges with the seperation from the above tables 
male = pd.merge(event[event.Gender=='Men'],medal_table[medal_table.Gender == 'Men'],on=['Edition','NOC','Gender'])
women = pd.merge(event[event.Gender=='Women'],medal_table[medal_table.Gender == 'Women'],on=['Edition','NOC','Gender'])

#merge the male and female so they are side by side 
medal=pd.merge(male,women, on=["Edition","NOC"])
medal =medal.drop(['Gender_x','Gender_y'],axis = 1)

#change column Names
column_names = ['Year', 'NOC','MaleMC','Male%MOC','Male%MBE','FemaleMC','Female%MOC','Female%MBE']
medal.columns = column_names

#create weighted column 
mw=(medal['Male%MOC']/medal['Male%MBE']).fillna(0)
ww=(medal['Female%MOC']/medal['Female%MBE']).fillna(0)

medal['W%Male']= mw/(mw+ww)
medal['W%Female']=ww/(mw+ww)
medal= medal.drop(['Male%MOC','Female%MOC'],axis=1)
medal.to_csv('OlympicMedal.csv')
medal


,Year,NOC,MaleMC,Male%MBE,FemaleMC,Female%MBE,W%Male,W%Female
0,1896,AUS,2,100.000000,0,0.000000,1.000000,0.000000
1,1896,AUT,5,100.000000,0,0.000000,1.000000,0.000000
2,1896,DEN,6,100.000000,0,0.000000,1.000000,0.000000
3,1896,FRA,11,100.000000,0,0.000000,1.000000,0.000000
4,1896,GBR,7,100.000000,0,0.000000,1.000000,0.000000
5,1896,GER,33,100.000000,0,0.000000,1.000000,0.000000
6,1896,GRE,52,100.000000,0,0.000000,1.000000,0.000000
7,1896,HUN,6,100.000000,0,0.000000,1.000000,0.000000
8,1896,SUI,3,100.000000,0,0.000000,1.000000,0.000000
9,1896,USA,20,100.000000,0,0.000000,1.000000,0.000000


In [10]:
#Creating a test file for practing dimple
medal_2008 = medal[medal.Year == 2008]
medal_2008.to_csv("OM_2008.csv")

In [11]:
#uses t2 above to count the total amount of events possible to win (male verus female
event_year = t2.groupby(level=0).apply(lambda x: x.sum()) \
               .apply(lambda x: 100*x/float(x.sum()), axis = 1) \
               .unstack().reset_index()
event_year = event_year.drop('level_0', axis=1)

#merges tables together to create one table
medal_table=pd.merge(coun_gend, event_year, how='inner', on=['Edition','Gender'])

#seperates all men from table into two tables and then merges with the seperation from the above tables 
male = pd.merge(event[event.Gender=='Men'],medal_table[medal_table.Gender == 'Men'],on=['Edition','NOC','Gender'])
women = pd.merge(event[event.Gender=='Women'],medal_table[medal_table.Gender == 'Women'],on=['Edition','NOC','Gender'])

#merge the male and female so they are side by side 
medal=pd.merge(male,women, on=["Edition","NOC"])

medal.columns
#change column Names
column_names = ['Year', 'NOC','GenderM','MaleMC','Male%MOC','Male%MBE','GenderW','FemaleMC','Female%MOC','Female%MBE']
medal.columns = column_names
medal['TMC'] = medal['MaleMC']+medal['FemaleMC']
medal[['TMC']] = medal[['TMC']].astype(int)

#create weighted column 
mw=(medal['Male%MOC']/medal['Male%MBE']).fillna(0)
ww=(medal['Female%MOC']/medal['Female%MBE']).fillna(0)

medal['W%Male']= mw/(mw+ww)
medal['W%Female']=ww/(mw+ww)
medal['WMC%M']= medal['W%Male']*(medal['MaleMC']+medal['FemaleMC'])
medal['WMC%W']= medal['W%Female']*(medal['MaleMC']+medal['FemaleMC'])
medal= medal.drop(['Male%MOC','Female%MOC'],axis=1)



women = pd.concat([medal['Year'], medal['NOC'],medal['GenderW'],medal['FemaleMC'],
                   medal['Female%MBE'],medal['W%Female'], medal['WMC%W'], medal['TMC']],
                  axis=1,keys=['Year',"NOC",'Gender',"MC","MBE",'W%','W_MC','TMC'])
men = pd.concat([medal['Year'], medal['NOC'],medal['GenderM'],medal['MaleMC'],
                 medal['Male%MBE'], medal['W%Male'], medal['WMC%M'],medal['TMC']],
                  axis=1,keys=['Year',"NOC","Gender","MC","MBE",'W%','W_MC','TMC'])

medal = pd.concat([women,men])

medal.to_csv('Olympics_MC.csv')
medal



,Year,NOC,Gender,MC,MBE,W%,W_MC,TMC
0,1896,AUS,Women,0,0.000000,0.000000,0.000000,2
1,1896,AUT,Women,0,0.000000,0.000000,0.000000,5
2,1896,DEN,Women,0,0.000000,0.000000,0.000000,6
3,1896,FRA,Women,0,0.000000,0.000000,0.000000,11
4,1896,GBR,Women,0,0.000000,0.000000,0.000000,7
5,1896,GER,Women,0,0.000000,0.000000,0.000000,33
6,1896,GRE,Women,0,0.000000,0.000000,0.000000,52
7,1896,HUN,Women,0,0.000000,0.000000,0.000000,6
8,1896,SUI,Women,0,0.000000,0.000000,0.000000,3
9,1896,USA,Women,0,0.000000,0.000000,0.000000,20


In [12]:
import matplotlib.pyplot as plt 
m = medal[medal['Gender'] == "Men"]
m = m[m["Year"]== 2008 ]
plt.scatter(m.TMC,m['W%'])
plt.show()
 
m = medal[medal['Gender'] == "Women"]
m = m[m["Year"]== 2008 ]
plt.scatter(m.TMC,m['W%'])
plt.show()

In [110]:
#This box makes a new file that only has countries in 2008
#it Also adds a column for continents 

medal_r = medal[medal['NOC'].isin(IOC_cont['NOC'])]
IOC_cont

#ADD cont to list 
cont_list = []
temp = "N/A"
for index,row in medal_r.iterrows():
    #try:
    #    cont = transformations.cn_to_ctn(row[1])
    #except: 
        try:
            temp = transformations.cc_to_cn(row[1])
        except:
            temp = "N/A"
        if temp != "N/A":
            cont = transformations.cn_to_ctn(temp)
        else:
            if row[1] == "BRU":
                cont = "Asia"
            elif row[1] == "GAM":
                cont = "Africa"
            elif row[1] == "ISV":
                cont = "North America"
            elif row[1] == "KOS":
                cont = "Europe"
            elif row[1] == "LBA":
                cont = "Africa"
            elif row[1] == "PLE":
                cont = "Asia"
            elif row[1] == "SSD":
                cont = "Europe"
            elif row[1] == "TPE":
                cont = "Asia"
            else:
                i = np.where(IOC_cont.NOC == row[1])[0][0]
                name = IOC_cont.iloc[i]['full_NOC']
                cont = transformations.cn_to_ctn(name)
                
        cont_list.append(cont)
    
medal_r['cont'] = cont_list

    
medal_r.to_csv('Olympics_MC.csv')

C:\Users\H.P\Anaconda2\lib\site-packages\ipykernel\__main__.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
OM.columns


Index([u'NOC', u'Gender', u'Event_gender', u'Medal', u'Edition'], dtype='object')

In [16]:
a= OM[OM.NOC == "USA"]
b= a[a.Edition == 2008]